In [1]:
!pip install -q kaggle

from google.colab import files
files.upload()  # Upload the kaggle.json file

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sowaadrahman","key":"ce2fab7c1ebaf7fc9160542dc7722a29"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d requiemonk/sentinel12-image-pairs-segregated-by-terrain

Dataset URL: https://www.kaggle.com/datasets/requiemonk/sentinel12-image-pairs-segregated-by-terrain
License(s): Attribution 4.0 International (CC BY 4.0)
100% 2.54G/2.55G [00:33<00:00, 113MB/s]
100% 2.55G/2.55G [00:33<00:00, 81.9MB/s]


In [6]:
import os
import zipfile

# Path to the downloaded ZIP file
zip_file_path = "sentinel12-image-pairs-segregated-by-terrain.zip"

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('dataset')

print("Extraction complete.")

# Delete the ZIP file after extraction
if os.path.exists(zip_file_path):
    os.remove(zip_file_path)
    print(f"{zip_file_path} deleted.")
else:
    print(f"{zip_file_path} not found.")


Extraction complete.
sentinel12-image-pairs-segregated-by-terrain.zip deleted.


In [7]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define image dimensions
IMAGE_SIZE = (224, 224)  # VGG16 and ResNet50 typically take 224x224 images
BATCH_SIZE = 128

# Path to dataset
dataset_path = "/content/dataset/v_2"

# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% of data for validation
)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Number of classes
num_classes = train_generator.num_classes

# Function to add classification layers to the pre-trained model
def build_model(base_model):
    # Freeze the convolutional base
    base_model.trainable = False

    # Add classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Load the VGG16 model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
vgg16_model = build_model(vgg16_base)

# Load the ResNet50 model
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
resnet50_model = build_model(resnet50_base)

# Train the models
# Training VGG16 model
vgg16_model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Training ResNet50 model
resnet50_model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Evaluate the models
vgg16_eval = vgg16_model.evaluate(validation_generator)
resnet50_eval = resnet50_model.evaluate(validation_generator)

print(f"VGG16 Evaluation - Loss: {vgg16_eval[0]}, Accuracy: {vgg16_eval[1]}")
print(f"ResNet50 Evaluation - Loss: {resnet50_eval[0]}, Accuracy: {resnet50_eval[1]}")


Found 25600 images belonging to 4 classes.
Found 6400 images belonging to 4 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


200/200 ━━━━━━━━━━━━━━━━━━━━ 200s 785ms/step - accuracy: 0.7102 - loss: 0.9681 - val_accuracy: 0.4314 - val_loss: 1.2940
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 160s 784ms/step - accuracy: 0.8909 - loss: 0.4023 - val_accuracy: 0.5197 - val_loss: 1.2746
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 172s 842ms/step - accuracy: 0.9200 - loss: 0.2834 - val_accuracy: 0.5472 - val_loss: 1.3300
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 190s 787ms/step - accuracy: 0.9346 - loss: 0.2311 - val_accuracy: 0.5727 - val_loss: 1.3094
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 171s 837ms/step - accuracy: 0.9437 - loss: 0.1983 - val_accuracy: 0.5708 - val_loss: 1.3636
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 192s 788ms/step - accuracy: 0.9491 - loss: 0.1732 - val_accuracy: 0.6042 - val_loss: 1.2713
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 160s 780ms/step - accuracy: 0.9535 - loss: 0.1587 - val_accuracy: 0.5977 - val_loss: 1.3614
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 160s 780ms/step - accuracy: 0.9568 - loss: 0.14